# ETL: автоматизация подготовки данных семинары

## Урок 2. Введение в подготовку данных для аналитиков. Таблицы фактов и таблицы измерений

1. Скачайте датасет fifа_s2.сsv. Проанализируйте его и определите, какие данные являются неполными. Удалите
ненужные колонки и недостающие значения.

2. Найдите в датафрейме полные дубликаты и удалите их. Значения могут быть одинаковыми, но написаны по-разному. Например, может отличаться размер регистра (заглавные и строчные буквы). Особое внимание уделить колонке с названиями команд.

3. Напишите функцию, которая добавит колонку с разбиением возраста по группам: до 20, от 20 до 30, от 30 до
36 и старше 36. Посчитайте количество футболистов в каждой категории.

In [1]:
import init_spark_env

In [31]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, udf
from pyspark.sql.types import StringType

In [32]:
# Инициализация Spark Session
spark = SparkSession.builder.appName("FIFA Data Preparation").getOrCreate()

In [33]:
spark

In [34]:
# Загрузка данных
df = spark.read.csv("fifa_s2.csv", header=True, inferSchema=True)
df

DataFrame[ID: int, Name: string, Age: int, Nationality: string, Overall: int, Potential: int, Club: string, Value: int, Wage: int, Preferred Foot: string, International Reputation: int, Skill Moves: int, Position: string, Joined: int, Contract Valid Until: string, Height: double, Weight: double, Release Clause: double]

In [35]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[ID: int, Name: string, Age: int, Nationality: string, Overall: int, Potential: int, Club: string, Value: int, Wage: int, Preferred Foot: string, International Reputation: int, Skill Moves: int, Position: string, Joined: int, Contract Valid Until: string, Height: double, Weight: double, Release Clause: double]>

In [36]:
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Overall: integer (nullable = true)
 |-- Potential: integer (nullable = true)
 |-- Club: string (nullable = true)
 |-- Value: integer (nullable = true)
 |-- Wage: integer (nullable = true)
 |-- Preferred Foot: string (nullable = true)
 |-- International Reputation: integer (nullable = true)
 |-- Skill Moves: integer (nullable = true)
 |-- Position: string (nullable = true)
 |-- Joined: integer (nullable = true)
 |-- Contract Valid Until: string (nullable = true)
 |-- Height: double (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Release Clause: double (nullable = true)



In [39]:
df.count()

2399

In [40]:
# Подсчет количества отсутствующих значений в каждой колонке
df_missing_values = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])

# Вывод количества отсутствующих значений
df_missing_values.show()

+---+----+---+-----------+-------+---------+----+-----+----+--------------+------------------------+-----------+--------+------+--------------------+------+------+--------------+
| ID|Name|Age|Nationality|Overall|Potential|Club|Value|Wage|Preferred Foot|International Reputation|Skill Moves|Position|Joined|Contract Valid Until|Height|Weight|Release Clause|
+---+----+---+-----------+-------+---------+----+-----+----+--------------+------------------------+-----------+--------+------+--------------------+------+------+--------------+
|  0|   0|  0|          7|      0|        0| 241|  257|   0|             0|                      10|          6|      21|     0|                 273|     0|     0|             3|
+---+----+---+-----------+-------+---------+----+-----+----+--------------+------------------------+-----------+--------+------+--------------------+------+------+--------------+



In [41]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

+--------+-----+
|AgeGroup|count|
+--------+-----+
|31 to 36|  328|
|Under 20|  270|
| Over 36|   14|
|20 to 30| 1746|
+--------+-----+



24/10/13 11:27:01 ERROR DAGScheduler: Failed to update accumulator 0 (org.apache.spark.api.python.PythonAccumulatorV2) for task 0
org.apache.spark.SparkException: EOF reached before Python server acknowledged
	at org.apache.spark.api.python.PythonAccumulatorV2.merge(PythonRDD.scala:751)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$updateAccumulators$1(DAGScheduler.scala:1694)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$updateAccumulators$1$adapted(DAGScheduler.scala:1685)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.updateAccumulators(DAGScheduler.scala:1685)
	at org.apache.spark.scheduler.DAGScheduler.handleTaskCompletion(DAGScheduler.scala:1838)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:3054)
	at o

In [42]:
spark.stop()